In [1]:
from new_helper import *
from super_helper import *

In [2]:
id = "06535"
DOMAIN = "test"

project_id = f"{id}@{DOMAIN}"

directory_path = "upload"

print(40*"-")



----------------------------------------


In [3]:


schema = get_schema() #super_helper.py

print(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

<Schema: ['abstract', 'created', 'creator', 'date', 'description', 'file_cid', 'format', 'full_text', 'language', 'metadata_cid', 'modified', 'project_id', 'publisher', 'subject', 'title']>
file path:  upload/Integration Test.ipynb
file name:  Integration Test.ipynb
file cid:  QmVk8gxENq6PThSGZTuk7RtbufR2XeN7LtXBj2LVTUoAdN
file metadata cid:  QmRxGqNkpdBVeq2GmfbK2ZbW1oZyX14VtiYRZnmSq6nnm7
file_key : file_1
joined_cids : QmVk8gxENq6PThSGZTuk7RtbufR2XeN7LtXBj2LVTUoAdN, QmRxGqNkpdBVeq2GmfbK2ZbW1oZyX14VtiYRZnmSq6nnm7
	Entering "create_contract"
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "create_contract"
	Entering "get_engine_receipts_address"
	Leaving "get_engine_receipts_address"
	Entering "set_account_detail"
None
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-01-03 20:54:39.583 | INFO     | super_helper:setup_index:288 - Opened existing index.


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
hash : b'8e6e2de8b491c0628e1abae4caa324989883addeeb70ca5d8fac15040aca569f'


2025-01-03 20:54:40.607 | INFO     | super_helper:index_metadata:98 - Metadata indexed successfully: QmRxGqNkpdBVeq2GmfbK2ZbW1oZyX14VtiYRZnmSq6nnm7


This is ix:  FileIndex(FileStorage('indexdir'), 'MAIN')
file path:  upload/Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf
file name:  Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf
file cid:  QmT7ximApXtExAedhzUHGHDwRismRzuKykpH4dgTvGrNZs
file metadata cid:  Qmdx66C5eMBgBw1NxB6hdxAc6v9g2ctGPZMPnYED4JYSYe
file_key : file_2
joined_cids : QmT7ximApXtExAedhzUHGHDwRismRzuKykpH4dgTvGrNZs, Qmdx66C5eMBgBw1NxB6hdxAc6v9g2ctGPZMPnYED4JYSYe
	Entering "set_account_detail"
None
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


...

('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
hash : b'9d6a9edfd44082a089b93070203440dd07c319e4715f06f9aee7e4e0681669c6'
This is ix:  FileIndex(FileStorage('indexdir'), 'MAIN')
file path:  upload/COVID19-MLSF--A-multi-task-learning-based-stock-market_2023_Expert-Systems-w.pdf
file name:  COVID19-MLSF--A-multi-task-learning-based-stock-market_2023_Expert-Systems-w.pdf
file cid:  QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
file metadata cid:  QmedM2sDbqRi5VmoWAUvwbuPrtSkLjHzYqor4FWK4223FV
file_key : file_4
joined_cids : QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, QmedM2sDbqRi5VmoWAUvwbuPrtSkLjHzYqor4FWK4223FV
	Entering "set_account_detail"
None
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-01-03 20:54:48.709 | INFO     | super_helper:setup_index:288 - Opened existing index.
2025-01-03 20:54:48.874 | INFO     | super_helper:index_metadata:98 - Metadata indexed successfully: QmedM2sDbqRi5VmoWAUvwbuPrtSkLjHzYqor4FWK4223FV


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
hash : b'4a614bf259de068c59bd6aa955ee2db1ee91b94492711e96a9f40ad064f59c16'
This is ix:  FileIndex(FileStorage('indexdir'), 'MAIN')
file path:  upload/Deep-learning-in-insurance--Accuracy-and-model-int_2023_Expert-Systems-with-.pdf
file name:  Deep-learning-in-insurance--Accuracy-and-model-int_2023_Expert-Systems-with-.pdf
file cid:  QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
file metadata cid:  QmeVGyHTArPSs2psP26Akwi69ZbGNN7J8Tri7JzMfSJsFN
file_key : file_5
joined_cids : QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, QmeVGyHTArPSs2psP26Akwi69ZbGNN7J8Tri7JzMfSJsFN
	Entering "set_account_detail"
None
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-01-03 20:54:51.760 | INFO     | super_helper:setup_index:288 - Opened existing index.
2025-01-03 20:54:51.899 | INFO     | super_helper:index_metadata:98 - Metadata indexed successfully: QmeVGyHTArPSs2psP26Akwi69ZbGNN7J8Tri7JzMfSJsFN


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
hash : b'd04003507c4f94b831cfee8fd94fa9db272707b675c21a8b716595704e366ac2'
This is ix:  FileIndex(FileStorage('indexdir'), 'MAIN')


In [4]:
account_detail = get_account_detail(project_id)
print(project_id, account_detail)


06535@test { "admin@test" : { "file_1" : "QmVk8gxENq6PThSGZTuk7RtbufR2XeN7LtXBj2LVTUoAdN, QmRxGqNkpdBVeq2GmfbK2ZbW1oZyX14VtiYRZnmSq6nnm7", "file_2" : "QmT7ximApXtExAedhzUHGHDwRismRzuKykpH4dgTvGrNZs, Qmdx66C5eMBgBw1NxB6hdxAc6v9g2ctGPZMPnYED4JYSYe", "file_3" : "QmZnh5Uuo7moZKMESFHpRjQFbbND5e3nArCzZg8uLARAgg, QmXgwRkPFxG2FWQTbVGgKsaEz7thbg4WLBcndsRfKUfeaa", "file_4" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, QmedM2sDbqRi5VmoWAUvwbuPrtSkLjHzYqor4FWK4223FV", "file_5" : "QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, QmeVGyHTArPSs2psP26Akwi69ZbGNN7J8Tri7JzMfSJsFN", "linked_user" : "crazy_nash@test", "project_metadata_cid" : "QmbmyecKUbwCDFPt6Uddx411bCvefLiRXPxHs5vnYz4XZm" } }


In [5]:
# recreate_index() #Controls the reset of the index manually

In [6]:

# Open an existing index (adjust the path to your index directory)
index_path = "indexdir"
index = open_dir(index_path)

# Perform a keyword search
keyword = "paper"

search_results, project_ids_with_cids = search_index(index, keyword)

# Now you can directly access each dictionary from 'search_results'
for dict in search_results:
    project_id = dict['project_id']
    print(dict['project_id'])
    file_cid = dict['file_cid']
    print(dict['file_cid'])
    metadata_cid = dict['metadata_cid']
    print(dict['metadata_cid'])
    
    project_details = get_account_detail(dict['project_id'])
    
    print(project_details)
    
    blockchain_data = json.loads(project_details)
    
    result = validate_file_cid(file_cid, blockchain_data)

    if result:
        print("File is valid.")
    else:
        print("File is invalid.")

2025-01-03 20:54:52.130 | INFO     | super_helper:search_index:116 - Starting keyword search...
2025-01-03 20:54:52.132 | INFO     | super_helper:search_index:117 - Keyword: 'paper'
2025-01-03 20:54:52.178 | INFO     | super_helper:search_index:128 - Search successful: Found 30 result(s).
2025-01-03 20:54:52.182 | INFO     | super_helper:search_index:130 - 1. Project Id: 06535@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, Metadata CID: QmSQGvPYaP8n53HALbGuAX5QfGu6zTFzpG6adWAsx5cAmW, Title: deep learning in insurance: accuracy and model interpretability using tabnet
2025-01-03 20:54:52.186 | INFO     | super_helper:search_index:130 - 2. Project Id: 06535@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, Metadata CID: QmfFa6RYjNWygojdBpFWzxYFW3Ea9Ldh4dNnAXteikNBFE, Title: deep learning in insurance: accuracy and model interpretability using tabnet
2025-01-03 20:54:52.189 | INFO     | super_helper:search_index:130 - 3. Project Id: 37887@test, File CID: QmV1

06535@test
QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
QmSQGvPYaP8n53HALbGuAX5QfGu6zTFzpG6adWAsx5cAmW
{ "admin@test" : { "file_1" : "QmVk8gxENq6PThSGZTuk7RtbufR2XeN7LtXBj2LVTUoAdN, QmRxGqNkpdBVeq2GmfbK2ZbW1oZyX14VtiYRZnmSq6nnm7", "file_2" : "QmT7ximApXtExAedhzUHGHDwRismRzuKykpH4dgTvGrNZs, Qmdx66C5eMBgBw1NxB6hdxAc6v9g2ctGPZMPnYED4JYSYe", "file_3" : "QmZnh5Uuo7moZKMESFHpRjQFbbND5e3nArCzZg8uLARAgg, QmXgwRkPFxG2FWQTbVGgKsaEz7thbg4WLBcndsRfKUfeaa", "file_4" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, QmedM2sDbqRi5VmoWAUvwbuPrtSkLjHzYqor4FWK4223FV", "file_5" : "QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, QmeVGyHTArPSs2psP26Akwi69ZbGNN7J8Tri7JzMfSJsFN", "linked_user" : "crazy_nash@test", "project_metadata_cid" : "QmbmyecKUbwCDFPt6Uddx411bCvefLiRXPxHs5vnYz4XZm" } }
File is valid.
06535@test
QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
QmfFa6RYjNWygojdBpFWzxYFW3Ea9Ldh4dNnAXteikNBFE
{ "admin@test" : { "file_1" : "QmVk8gxENq6PThSGZTuk7RtbufR2XeN7LtXBj2LVTUoAdN, QmRxGqNkpdBVeq2Gmfb

2025-01-03 20:54:52.508 | DEBUG    | super_helper:validate_file_cid:192 - Validating CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-03 20:54:52.509 | DEBUG    | super_helper:validate_file_cid:200 - CID QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW found in file_4.
2025-01-03 20:54:52.526 | DEBUG    | super_helper:validate_file_cid:192 - Validating CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-03 20:54:52.528 | DEBUG    | super_helper:validate_file_cid:200 - CID QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW found in file_4.
2025-01-03 20:54:52.544 | DEBUG    | super_helper:validate_file_cid:192 - Validating CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-03 20:54:52.545 | DEBUG    | super_helper:validate_file_cid:200 - CID QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW found in file_4.
2025-01-03 20:54:52.560 | DEBUG    | super_helper:validate_file_cid:192 - Validating CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-03 20:54:52.561 | 

{ "admin@test" : { "file_1" : "QmVk8gxENq6PThSGZTuk7RtbufR2XeN7LtXBj2LVTUoAdN, QmRxGqNkpdBVeq2GmfbK2ZbW1oZyX14VtiYRZnmSq6nnm7", "file_2" : "QmT7ximApXtExAedhzUHGHDwRismRzuKykpH4dgTvGrNZs, QmSCF9dc1p25xuKapk8kem2jJj3rG1U5L6vb72nhkMpr3h", "file_3" : "QmZnh5Uuo7moZKMESFHpRjQFbbND5e3nArCzZg8uLARAgg, QmXgwRkPFxG2FWQTbVGgKsaEz7thbg4WLBcndsRfKUfeaa", "file_4" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, QmSPumufDMLPk1x7mUY6QCZaaCooNZmDjYqQsd2UQTQxti", "file_5" : "QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, Qmb2oVa1Ukvh9mUgih8EoqgUeAPPUnLY73LHGenAH8o3Nc", "project_metadata_cid" : "QmT1cPcYNcGntxCc2goVmkJuQG9EdVthjvG6SVNs7WCXc8" } }
File is valid.
37887@test
QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
QmbDa6tBa5HBWikEf2R74kuNzBAMKNhgc44HbcHajbNswT
{ "admin@test" : { "file_1" : "QmVk8gxENq6PThSGZTuk7RtbufR2XeN7LtXBj2LVTUoAdN, QmRxGqNkpdBVeq2GmfbK2ZbW1oZyX14VtiYRZnmSq6nnm7", "file_2" : "QmT7ximApXtExAedhzUHGHDwRismRzuKykpH4dgTvGrNZs, QmSCF9dc1p25xuKapk8kem2jJj3rG1U5L6vb72nhkMpr3h", "

In [7]:
result = fetch_project_details(file_cid, blockchain_data)
logger.info(f"Result: {result}")

2025-01-03 20:54:52.645 | DEBUG    | super_helper:fetch_project_details:223 - Validating and fetching details for CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-03 20:54:52.646 | DEBUG    | super_helper:fetch_project_details:231 - File CID QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW found under file_4.
2025-01-03 20:54:52.648 | INFO     | __main__:<module>:2 - Result: {'is_valid': True, 'project_metadata_cid': 'QmbmyecKUbwCDFPt6Uddx411bCvefLiRXPxHs5vnYz4XZm', 'linked_user': 'crazy_nash@test'}


In [ ]:
result = fetch_project_details(file_cid, blockchain_data)

if result["is_valid"]:
    project_metadata_cid = result["project_metadata_cid"]
    linked_user = result["linked_user"]
    
    # Use the returned values
    logger.info(f"Project Metadata CID: {project_metadata_cid}")
    logger.info(f"Linked User: {linked_user}")
    
    # Perform other actions with the data
    if project_metadata_cid:
        # Process project metadata CID
        logger.debug(f"Processing project metadata CID: {project_metadata_cid}")
    if linked_user:
        # Process linked user
        logger.debug(f"Processing linked user: {linked_user}")
else:
    logger.warning("File CID is invalid. No project metadata or linked user available.")




2025-01-03 21:12:59.100 | DEBUG    | super_helper:fetch_project_details:223 - Validating and fetching details for CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-03 21:12:59.103 | DEBUG    | super_helper:fetch_project_details:231 - File CID QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW found under file_4.
2025-01-03 21:12:59.105 | INFO     | __main__:<module>:8 - Project Metadata CID: QmbmyecKUbwCDFPt6Uddx411bCvefLiRXPxHs5vnYz4XZm
2025-01-03 21:12:59.107 | INFO     | __main__:<module>:9 - Linked User: crazy_nash@test
2025-01-03 21:12:59.108 | DEBUG    | __main__:<module>:14 - Processing project metadata CID: QmbmyecKUbwCDFPt6Uddx411bCvefLiRXPxHs5vnYz4XZm
2025-01-03 21:12:59.109 | DEBUG    | __main__:<module>:17 - Processing linked user: crazy_nash@test


In [13]:
user = get_account_detail(linked_user)
user = json.loads(user)
print(user)

{'admin@test': {'linked_project': '06535@test', 'user_json_ld_cid': 'QmVQLbPmAqVYWPT1x9niJEC3w1YK5zxXYJPC8FraoasHuU'}}


In [14]:
user = get_account_detail(linked_user)
user = json.loads(user)  # Convert the JSON string to a Python dictionary

# Accessing user_json_ld_cid
user_json_ld_cid = user.get("admin@test", {}).get("user_json_ld_cid", None)

# Check if the value exists and use it
if user_json_ld_cid:
    print(f"user_json_ld_cid: {user_json_ld_cid}")
else:
    print("user_json_ld_cid not found.")


user_json_ld_cid: QmVQLbPmAqVYWPT1x9niJEC3w1YK5zxXYJPC8FraoasHuU


In [11]:
project_metadata = download_json_from_ipfs(project_metadata_cid)
print(project_metadata)

{'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This study explores the impact of quantum computing in supply chain management, focusing on biodiversity.', 'schema:endDate': '2026-07-23', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'National Science Foundation'}, 'schema:keywords': ['quantum computing', 'supply chain management', 'biodiversity'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Lagos, Nigeria'}, 'schema:name': 'Investigating the Role of quantum computing in supply chain management', 'schema:startDate': '2023-07-03'}
